In [1]:
pip install transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 34.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
emotion_labels = [
    "admiration", "amusement", "anger", "annoyance", "approval",
    "caring", "confusion", "curiosity", "desire", "disappointment",
    "disapproval", "disgust", "embarrassment", "excitement", "fear",
    "gratitude", "grief", "joy", "love", "nervousness",
    "optimism", "pride", "realization", "relief", "remorse",
    "sadness", "surprise", "neutral"
]

In [3]:
Cheat_sheet = {
    "admiration": ["praise", "acknowledge"],
    "amusement": ["joke", "lighten"],
    "anger": ["calm", "de_escalate"],
    "annoyance": ["empathize", "soothe"],
    "approval": ["agree", "validate"],
    "caring": ["support", "nurture"],
    "confusion": ["clarify", "explain"],
    "curiosity": ["inform", "explore"],
    "desire": ["encourage", "motivate"],
    "disappointment": ["empathize", "console"],
    "disapproval": ["caution", "advise"],
    "disgust": ["distance", "warn"],
    "embarrassment": ["reassure", "comfort"],
    "excitement": ["celebrate", "energize"],
    "fear": ["calm", "reassure"],
    "gratitude": ["acknowledge", "reciprocate"],
    "grief": ["comfort", "validate"],
    "joy": ["celebrate", "share"],
    "love": ["affirm", "support"],
    "nervousness": ["reassure", "encourage"],
    "optimism": ["encourage", "uplift"],
    "pride": ["acknowledge", "praise"],
    "realization": ["explain", "highlight"],
    "relief": ["affirm", "reassure"],
    "remorse": ["acknowledge", "console"],
    "sadness": ["empathize", "validate"],
    "surprise": ["explain", "react"],
    "neutral": ["acknowledge", "respond"]
}


In [4]:
import json

with open("/kaggle/input/notebookba7deda072/myprompts.json", "r") as f:
    myprompts = json.load(f)

In [5]:
import random

def myprompt_examples(emotion, n=6):
    if emotion not in myprompts:
        return ""

    examples = random.sample(
        myprompts[emotion],
        n
    )

    myprompt_text = ""
    for ex in examples:
        myprompt_text += f"""
User: {ex['User']}
Assistant: {ex['Assistant']}
"""

    return myprompt_text.strip()

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16,
    device_map="auto"
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [7]:
from peft import LoraConfig, get_peft_model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lora_config = LoraConfig(
    r=8,                    
    lora_alpha=32,             
    target_modules=["q_proj", "v_proj"],  
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
for name, param in model.named_parameters():
    param.requires_grad = "lora" in name

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Trainable parameters:", trainable_params)

Trainable parameters: 3407872


In [8]:
from datasets import Dataset




flat_data = []
for req_emotion in emotion_labels:
    for item in myprompts.get(req_emotion, []):  
       prompt_text = f"User: {item['User']}\nAssistant:"
       completion_text = f" {item['Assistant']}"
       flat_data.append({"prompt": prompt_text, "completion": completion_text})


dataset = Dataset.from_list(flat_data)

In [9]:
dataset = Dataset.from_list(flat_data)
print(f"Total samples: {len(dataset)}")

Total samples: 162


In [10]:

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

print("PAD token:", tokenizer.pad_token)
print("PAD token ID:", tokenizer.pad_token_id)

PAD token: </s>
PAD token ID: 2


In [11]:
def tokenize_function(examples):
    texts = [p + c for p, c in zip(examples["prompt"], examples["completion"])]
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=256,
    )
 
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized
tokenized_dataset = dataset.map(tokenize_function, batched=True)
print(tokenized_dataset[0])

Map:   0%|          | 0/162 [00:00<?, ? examples/s]

{'prompt': 'User: I’ve been waking up at 6 a.m. every day for the past month to study, even when I feel tired.\nAssistant:', 'completion': ' That’s genuinely admirable. Consistency like that isn’t easy, especially when motivation fades. Showing up every day proves real discipline and long-term thinking — something many people struggle to build.', 'input_ids': [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1247, 28747, 315, 28809, 333, 750, 275, 1288, 582, 438, 28705, 28784, 264, 28723, 28719, 28723, 1012, 1370

In [12]:
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq


In [13]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True,
    return_tensors="pt"
)


In [14]:
training_args = TrainingArguments(
    output_dir="./lora_mistral",     
    per_device_train_batch_size=1,   
    gradient_accumulation_steps=8,    
    num_train_epochs=3,             
    learning_rate=5e-5, 
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    fp16=True,                      
    logging_steps=3,
    save_strategy="steps",
    save_steps=200,          
    save_total_limit=3,
    report_to="none"                   
)


In [15]:
from transformers import Trainer

class CausalLMTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        if labels is None:
            raise ValueError("Labels are missing! Make sure your dataset produces 'labels'.")
        inputs["labels"] = labels.long()
        
        outputs = model(**inputs)
        loss = getattr(outputs, "loss", None)
        
        if loss is None:
            import torch
            logits = outputs.logits
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = inputs["labels"][..., 1:].contiguous()
            loss_fct = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
            loss = loss_fct(
                shift_logits.view(-1, shift_logits.size(-1)),
                shift_labels.view(-1)
            )
        return (loss, outputs) if return_outputs else loss


In [16]:
trainer = CausalLMTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)



/tmp/ipykernel_24/183362814.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CausalLMTrainer.__init__`. Use `processing_class` instead.
  trainer = CausalLMTrainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


In [17]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Step,Training Loss
3,43.912900
6,43.437800
9,42.956600
12,42.309900
15,42.334600
18,42.243300
21,31.183700
24,41.627600
27,41.681200
30,41.516600


TrainOutput(global_step=63, training_loss=40.27432359967913, metrics={'train_runtime': 585.3168, 'train_samples_per_second': 0.83, 'train_steps_per_second': 0.108, 'total_flos': 5310623281840128.0, 'train_loss': 40.27432359967913, 'epoch': 3.0})

In [18]:
model.save_pretrained("/kaggle/working/emo_lora_adapter")
tokenizer.save_pretrained("/kaggle/working/emo_lora_adapter")

('/kaggle/working/emo_lora_adapter/tokenizer_config.json',
 '/kaggle/working/emo_lora_adapter/special_tokens_map.json',
 '/kaggle/working/emo_lora_adapter/chat_template.jinja',
 '/kaggle/working/emo_lora_adapter/tokenizer.model',
 '/kaggle/working/emo_lora_adapter/added_tokens.json',
 '/kaggle/working/emo_lora_adapter/tokenizer.json')

In [19]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    torch_dtype=torch.float16,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, "./emo_lora_adapter")
tokenizer = AutoTokenizer.from_pretrained("./emo_lora_adapter")


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]